In [48]:
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('punkt')

def sentence_to_wordlist(sentence, remove_stopwords=False):
    # 1. Remove non-letters
    sentence_text = re.sub(r'[^\w\s]','', sentence)
    # 2. Remove all numbers
    sentence_text = re.sub(r'[0-9]+', '', sentence_text)
    # 3. Convert words to lower case and split them
    words = sentence_text.lower().split()
    # 4. Stemming
    words = [stemmer.stem(w) for w in words] 
    # 5. Lemmatizing
    words = [lemmatizer.lemmatize(word) for word in words]
    # 6. Return a list of words
    return(words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Leonardo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leonardo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
# Find the max length sentence
def find_max_length_sentence(sentence):
    max_length = 0
    for i in sentence:
#         length = len(sentence_to_wordlist(i))
        length = len(i)
        if max_length < length:
            max_length = length
    return max_length

In [101]:
import pandas as pd

df = pd.read_csv('../data/processed/headlines_cl_sent.csv')

In [29]:
sentence = df['clean_headline']
seq_length = find_max_length_sentence(sentence)

In [19]:
# from tensorflow.keras.models import model_from_json
from gensim.models import Word2Vec
from collections import Counter
import re
from keras.preprocessing.sequence import pad_sequences
# # Model reconstruction from JSON file
# with open('../models/biasly_model_architecture.json', 'r') as f:
#     loaded_model = model_from_json(f.read())

# # Load weights into the new model
# loaded_model.load_weights('../models/biasly_model_weights.h5')

# load model
wv_model = Word2Vec.load('../models/model.bin')
word_vectors = wv_model.wv
words = list(wv_model.wv.vocab)
# Calling init_sims will make the model will be better for memory
# if we don't want to train the model over and over again
wv_model.init_sims(replace=True)

#n_words = print(len(words))

print("Number of word vectors: {}".format(len(word_vectors.vocab)))

Number of word vectors: 3863


Using TensorFlow backend.


In [10]:
wv_dim = 100
num_words = len(word_vectors.vocab)
vocab = Counter(words)

word_index = {t[0]: i+1 for i,t in enumerate(vocab.most_common(num_words-1))}

In [77]:
def predict_sentences():
    #print("Please input a sentence: ")
    pred_text = sentence_to_wordlist(input())
#     pred_text = input()
    pred_sequences = [word_index.get(t, 0) for t in pred_text]
    pred_data = pad_sequences([pred_sequences], maxlen=seq_length, padding="post", truncating="post")
    prediction = loaded_model.predict(pred_data)
    return "LSTM Model thinks the sentence is %.2f%% biased."%(prediction[0][0] * 100)
    #    return "%.2f%%"%(prediction[0][0] *100)

In [82]:
print(predict_sentences())

 All boys are kind


[[0.6306776]]


In [108]:
import ast
df['clean_hl_words'] = df['clean_hl_words'].apply(lambda row: ast.literal_eval(row))
df['clean_hl_words'][1000]

['beautiful',
 'die',
 'catastrophic',
 'injury',
 'fall',
 'tree',
 'branch',
 'kill',
 'family',
 'walk']

In [104]:
df['clean_headline'][1000]

"['beautiful', 'girl', 'die', 'catastrophic', 'injury', 'fall', 'tree', 'branch', 'kill', 'family', 'walk']"

In [120]:
#Two lists  of words that are used when a man or woman is present, based on Danielle Sucher's https://github.com/DanielleSucher/Jailbreak-the-Patriarchy
male_words=set(['guy','spokesman','chairman',"men's",'men','him',"he's",'his','boy','boyfriend','boyfriends','boys','brother','brothers','dad','dads','dude','father','fathers','fiance','gentleman','gentlemen','god','grandfather','grandpa','grandson','groom','he','himself','husband','husbands','king','male','man','mr','nephew','nephews','priest','prince','son','sons','uncle','uncles','waiter','widower','widowers'])
female_words=set(['heroine','spokeswoman','chairwoman',"women's",'actress','women',"she's",'her','aunt','aunts','bride','daughter','daughters','female','fiancee','girl','girlfriend','girlfriends','girls','goddess','granddaughter','grandma','grandmother','herself','ladies','lady','lady','mom','moms','mother','mothers','mrs','ms','niece','nieces','priestess','princess','queens','she','sister','sisters','waitress','widow','widows','wife','wives','woman', 'beautiful'])

In [121]:
def gender_the_sentence(sentence_words):
    mw_length=len(male_words.intersection(sentence_words))
    fw_length=len(female_words.intersection(sentence_words))

    if mw_length>0 and fw_length==0:
        gender='male'
    elif mw_length==0 and fw_length>0: 
        gender='female'
    elif mw_length>0 and fw_length>0: 
        gender='both'
    else:
        gender='none'
    return gender

In [122]:
sentence = set([w.lower() for w in df['clean_hl_words'][1000]])

In [123]:
sentence

{'beautiful',
 'branch',
 'catastrophic',
 'die',
 'fall',
 'family',
 'injury',
 'kill',
 'tree',
 'walk'}

In [124]:
gender_the_sentence(sentence)

'female'